In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/Emotion Recogition/code/python_files

/content/drive/My Drive/Emotion Recogition/code/python_files


In [3]:
import numpy as np
import os
import sys
import pandas as pd

import wave
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten,Dropout,GlobalAveragePooling2D,MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from scipy import signal
import matplotlib.pyplot as plot
from sklearn.preprocessing import LabelEncoder
from keras import backend as K

Using TensorFlow backend.


In [0]:
from helper import *

In [0]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [0]:
import pickle
with open(code_path + '/../data/'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [7]:
!pip install python_speech_features  

In [0]:
from python_speech_features import mfcc
from python_speech_features import logfbank
from python_speech_features import fbank
import scipy.io.wavfile as wav
from sklearn.preprocessing import scale

In [10]:
X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used:
        fbank_feat = logfbank(ses_mod['signal'],nfilt=40)
        Sxx = np.transpose(fbank_feat)
        Sxx = pad_sequence_into_array( Sxx,maxlen=300,value=0)
        Sxx = scale(Sxx)
       
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

625
2034
(2034, 40, 300)
(625, 40, 300)


In [0]:
import matplotlib.pyplot as plt

In [0]:
X_train = []
X_test = []
Y_train = []
Y_test = []
fs = 16e3
counter = 0
counter1 = 0
for ses_mod in data2:
    if 'impro' in ses_mod['id'] and ses_mod['emotion'] in emotions_used:
        f, t, Sxx = signal.spectrogram(ses_mod['signal'], fs, nperseg=400,noverlap=200)
        Sxx = pad_sequence_into_array(Sxx,maxlen=300,value=0)
        
        if ses_mod['id'][:5]=="Ses05":
            counter+=1
            X_test.append(Sxx)
            Y_test.append(ses_mod['emotion'])
        else:
            counter1+=1
            X_train.append(Sxx)
            Y_train.append(ses_mod['emotion'])
        
print(counter)
print(counter1)

X_test = np.array(X_test)
X_train = np.array(X_train)
print(X_train.shape)
print(X_test.shape)

625
2034
(2034, 201, 300)
(625, 201, 300)


In [0]:
y  = pd.get_dummies(Y_train+Y_test)
y_train = y[0:len(Y_train)]
y_test = y[len(Y_train):]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed

In [0]:
X_train = X_train.reshape(-1,40,300,1)
X_test = X_test.reshape(-1,40,300,1)

In [0]:
def make_model():
    in_layer = Input(shape=(40, 300,1))
    x = Conv2D(32,(5,5), activation = 'elu')(in_layer)  
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(64,(5,5), activation = 'elu')(x)         
    x = Dropout(0.5)(x)
    x = Conv2D(128, (5,5))(x)                           
    x = GlobalAveragePooling2D()(x)                     
    x = Dense(64,activation='elu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(4, activation = "softmax")(x) # softmax output
    model = Model(inputs = in_layer, outputs=output_layer)
    return model

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = categorical_focal_loss(gamma=.25, alpha=1.),
                    metrics=["accuracy"])
Y_label = ( [np.where(r==1)[0][0] for r in y_train] )
early_stopping_monitor = EarlyStopping( monitor='val_loss',patience=20,verbose=1)
m_check = keras.callbacks.ModelCheckpoint(filepath = './lstm_encoder.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check,early_stopping_monitor],class_weight = class_weight.compute_class_weight("balanced", np.unique(Y_label), Y_label))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 2034 samples, validate on 625 samples
Epoch 1/100
2034/2034 [==============================] - 23s 12ms/step - loss: 1.1670 - acc: 0.4243 - val_loss: 1.0963 - val_acc: 0.4896

Epoch 00001: val_acc improved from -inf to 0.48960, saving model to ./cnn_focalloss.h5
Epoch 2/100
2034/2034 [==============================] - 16s 8ms/step - loss: 1.0155 - acc: 0.4892 - val_loss: 1.0905 - val_acc: 0.4640

Epoch 00002: val_acc did not improve from 0.48960
Epoch 3/100
2034/2034 [==============================] - 16s 8ms/step - loss: 0.9930 - acc: 0.5123 - val_loss: 0.9920 - val_acc: 0.5104

Epoch 00003: val_acc improved from 0.48960 to 0.51040, saving model to ./cnn_focalloss.h5
Epoch 4/100
2034/2034 [==============================] - 15s 8ms/step - loss: 0.9523 - acc: 0.5334 - val_loss: 1.0058 - val_acc: 0.5296

Epoch 00004: val_acc improved from 0.51040 to 0.52960, saving model to ./cnn_focalloss.h5
Epoch 5/100
2034/2034 [==============================] - 15s 8ms/step - loss: 0.9483 - 

In [0]:
model.save_weights('focalweight_complete.h5')

In [0]:
y_pred = model.predict(X_test)

In [0]:
from sklearn.metrics import classification_report, accuracy_score

In [0]:
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])) :
    if y_pred[i][j]==max(y_pred[i]) :
      y_pred[i][j] = 1
    else:
      y_pred[i][j]=0

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.09      0.23      0.13        31
           1       0.65      0.42      0.51       174
           2       0.62      0.72      0.67       287
           3       0.71      0.53      0.61       133

   micro avg       0.57      0.57      0.57       625
   macro avg       0.52      0.47      0.48       625
weighted avg       0.62      0.57      0.58       625
 samples avg       0.57      0.57      0.57       625



In [0]:
best_model = load_model('./cnn_focalloss.h5')

ValueError: ignored

In [0]:
y_pred = best_model.predict(X_test)

In [0]:
from sklearn.metrics import classification_report, accuracy_score

In [0]:
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])) :
    if y_pred[i][j]==max(y_pred[i]) :
      y_pred[i][j] = 1
    else:
      y_pred[i][j]=0

In [0]:
print(y_pred)

[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.34      0.45      0.39        31
           1       0.65      0.48      0.55       174
           2       0.65      0.78      0.71       287
           3       0.77      0.64      0.70       133

   micro avg       0.65      0.65      0.65       625
   macro avg       0.60      0.59      0.59       625
weighted avg       0.66      0.65      0.65       625
 samples avg       0.65      0.65      0.65       625



In [0]:
accuracy_score(y_pred, y_test)

0.6512

In [0]:
import theano.tensor as T
import theano
import numpy as np
from theano.compile.ops import as_op


@as_op(itypes=[theano.tensor.ivector],
       otypes=[theano.tensor.ivector])
def numpy_unique(a):
    return np.unique(a)


def lda_loss(n_components, margin):
    """
    The main loss function (inner_lda_objective) is wrapped in this function due to
    the constraints imposed by Keras on objective functions
    """
    def inner_lda_objective(y_true, y_pred):
        """
        It is the loss function of LDA as introduced in the original paper. 
        It is adopted from the the original implementation in the following link:
        https://github.com/CPJKU/deep_lda
        Note: it is implemented by Theano tensor operations, and does not work on Tensorflow backend
        """
        r = 1e-4

        # init groups
        yt = T.cast(y_true.flatten(), "int32")
        groups = numpy_unique(yt)

        def compute_cov(group, Xt, yt):
            Xgt = Xt[T.eq(yt, group).nonzero()[0], :]
            Xgt_bar = Xgt - T.mean(Xgt, axis=0)
            m = T.cast(Xgt_bar.shape[0], 'float32')
            return (1.0 / (m - 1)) * T.dot(Xgt_bar.T, Xgt_bar)

        # scan over groups
        covs_t, updates = theano.scan(fn=compute_cov, outputs_info=None,
                                      sequences=[groups], non_sequences=[y_pred, yt])

        # compute average covariance matrix (within scatter)
        Sw_t = T.mean(covs_t, axis=0)

        # compute total scatter
        Xt_bar = y_pred - T.mean(y_pred, axis=0)
        m = T.cast(Xt_bar.shape[0], 'float32')
        St_t = (1.0 / (m - 1)) * T.dot(Xt_bar.T, Xt_bar)

        # compute between scatter
        Sb_t = St_t - Sw_t

        # cope for numerical instability (regularize)
        Sw_t += T.identity_like(Sw_t) * r

        # return T.cast(T.neq(yt[0], -1), 'float32')*T.nlinalg.trace(T.dot(T.nlinalg.matrix_inverse(St_t), Sb_t))

        # compute eigenvalues
        evals_t = T.slinalg.eigvalsh(Sb_t, Sw_t)

        # get eigenvalues
        top_k_evals = evals_t[-n_components:]

        # maximize variance between classes
        # (k smallest eigenvalues below threshold)
        thresh = T.min(top_k_evals) + margin
        top_k_evals = top_k_evals[(top_k_evals <= thresh).nonzero()]
        costs = T.mean(top_k_evals)

        return -costs

    return inner_lda_objective

In [0]:
model  =  make_model()
model.compile(optimizer = "adam", loss = lda_loss(3,1),
                    metrics=["accuracy"])
m_check = keras.callbacks.ModelCheckpoint(filepath = './cnn_spectrogram1.h5', monitor='val_acc', save_best_only=True, mode='max', verbose=1 )
hist = model.fit(X_train, y_train, 
                 batch_size=32, validation_data=(X_test,y_test),nb_epoch=100, verbose=1, shuffle = True,callbacks=[m_check] 
                )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


AttributeError: ignored

In [0]:
model = best_model

In [0]:
from keras.models import Model

layer_name = 'global_average_pooling2d_7'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
output_train = intermediate_layer_model.predict(X_train)
output_test =  intermediate_layer_model.predict(X_test)


In [0]:
print(output_train.shape)
print(output_test.shape)
print(y_train.shape)

(2034, 128)
(625, 128)
(2034, 4)


In [0]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(Y_train)
y_test = labelencoder.transform(Y_test)

In [0]:
y_train.shape

(2034,)

In [0]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt

In [0]:
xg_model = XGBClassifier(n_estimators=100,learning_rate=0.2,max_depth=7,objective = 'multi:softmax',
                       num_class=4,n_jobs=-1)

In [0]:
xg_model.fit(output_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, num_class=4, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [0]:
plot_importance(xg_model)
plt.show()

In [0]:
thresholds = np.sort(xg_model.feature_importances_)

In [0]:
# select features using threshold
selection = SelectFromModel(xg_model, threshold=0.005, prefit=True)
X_train = selection.transform(output_train)
X_test = selection.transform(output_test)

In [0]:
xg_model1 = XGBClassifier(n_estimators=100,learning_rate=0.2,max_depth=7,objective = 'multi:softmax',
                       num_class=4,n_jobs=-1)

In [0]:
xg_model1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, num_class=4, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [0]:
y_pred = xg_model1.predict(X_test)

In [0]:
X_train.shape

(2034, 114)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.20      0.26      0.22        31
           1       0.64      0.44      0.52       174
           2       0.63      0.69      0.66       287
           3       0.64      0.71      0.67       133

   micro avg       0.60      0.60      0.60       625
   macro avg       0.53      0.52      0.52       625
weighted avg       0.61      0.60      0.60       625



In [0]:
y_pred = xg_model.predict(output_test)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.23      0.29      0.26        31
           1       0.62      0.44      0.51       174
           2       0.63      0.68      0.65       287
           3       0.63      0.72      0.67       133

   micro avg       0.60      0.60      0.60       625
   macro avg       0.53      0.53      0.52       625
weighted avg       0.61      0.60      0.60       625



In [0]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0,n_jobs=-1)
#classifier.fit(output_train, y_train)

In [0]:
y_pred = classifier.predict(output_test)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.24      0.26      0.25        31
           1       0.63      0.45      0.52       174
           2       0.65      0.72      0.68       287
           3       0.67      0.74      0.70       133

   micro avg       0.63      0.63      0.63       625
   macro avg       0.55      0.54      0.54       625
weighted avg       0.63      0.63      0.62       625



In [0]:
from mlxtend.evaluate import confusion_matrix
m = confusion_matrix(y_target=y_test, 
                      y_predicted=y_pred, 
                      binary=False)
print(m)

In [0]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 40,None],
 'max_features': ['auto', 'sqrt'],
 'n_estimators': [80,100,150,400]}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(output_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_


In [0]:
best_parameters

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'auto',
 'n_estimators': 80}

In [0]:
best_accuracy

0.6647000983284169

In [0]:
classifier = RandomForestClassifier(n_estimators = 80, bootstrap=True,criterion = 'entropy', max_depth=20,max_features='auto',n_jobs=-1)
classifier.fit(output_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
y_pred = classifier.predict(output_test)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.24      0.26      0.25        31
           1       0.64      0.43      0.51       174
           2       0.64      0.73      0.68       287
           3       0.66      0.73      0.70       133

   micro avg       0.62      0.62      0.62       625
   macro avg       0.55      0.54      0.53       625
weighted avg       0.62      0.62      0.62       625



In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(output_train)
X_test_scaled = sc.transform(output_test)

In [0]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
#classifier.fit(X_train_scaled, y_train)

In [0]:
y_pred = classifier.predict(X_test_scaled)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.30      0.23      0.26        31
           1       0.65      0.41      0.50       174
           2       0.62      0.76      0.69       287
           3       0.67      0.72      0.70       133

   micro avg       0.63      0.63      0.63       625
   macro avg       0.56      0.53      0.54       625
weighted avg       0.62      0.63      0.61       625



In [0]:
# Applying Grid Search to find the best model and the best parameters
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train_scaled, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [0]:
best_parameters

{'C': 1, 'kernel': 'linear'}

In [0]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', C=1)
classifier.fit(X_train_scaled, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = classifier.predict(X_test_scaled)

In [0]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.38      0.42      0.40        31
           1       0.74      0.36      0.49       174
           2       0.61      0.78      0.69       287
           3       0.65      0.70      0.67       133

   micro avg       0.63      0.63      0.63       625
   macro avg       0.60      0.56      0.56       625
weighted avg       0.65      0.63      0.61       625



In [0]:
import pickle
with open(filename, ‘wb’) as f:
    pickle.dump(y_train, f)

SyntaxError: ignored